In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7

In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import open3d as o3d

In [ ]:
# Set dataset path
dataset_path = '/content/drive/MyDrive/DSC 210 Final Project/Dataset'

# Load images
images = []
image_paths = sorted(glob.glob(os.path.join(dataset_path, '*.jpg')))  # Adjust file extension if needed
for img_path in image_paths:
    img = cv2.imread(img_path)
    images.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
print(f"Loaded {len(images)} images.")

Loaded 47 images.


In [ ]:
# Set the desired resolution
desired_width = 750
desired_height = 1000
desired_resolution = (desired_width, desired_height)

# Downscale images to the specific resolution
downscaled_images = []

for img in images:
    # Resize image to the desired resolution
    resized_img = cv2.resize(img, desired_resolution, interpolation=cv2.INTER_AREA)
    downscaled_images.append(resized_img)

print(f"Images resized to {desired_width}x{desired_height} resolution.")
images = downscaled_images


Images resized to 750x1000 resolution.


In [ ]:
# Feature Detection with SIFT with PLOT

sift = cv2.SIFT_create()
keypoints_list, descriptors_list = [], []

# Iterate through each image for feature detection
for i, img in enumerate(images):
    keypoints, descriptors = sift.detectAndCompute(img, None)
    keypoints_list.append(keypoints)
    descriptors_list.append(descriptors)

    # Visualize the detected keypoints
    img_with_keypoints = cv2.drawKeypoints(
        img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    )

    # Display the image with keypoints
    plt.figure(figsize=(8, 6))
    plt.title(f"Feature Detection using SIFT for Image {i}")
    plt.imshow(cv2.cvtColor(img_with_keypoints, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

print("Feature detection and visualization complete.")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Feature Matching

bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)  # Set crossCheck to False for knnMatch
matches = []
ratio_thresh = 0.75  # Lowe's ratio test threshold

# Iterate through image pairs for matching and visualization
for i in range(len(images) - 1):
    # Perform kNN matching
    knn_matches = bf.knnMatch(descriptors_list[i], descriptors_list[i + 1], k=2)

    # Apply Lowe's ratio test
    good_matches = []
    for m, n in knn_matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    matches.append(good_matches)

    # Draw the good matches
    img_matches = cv2.drawMatches(
        images[i], keypoints_list[i],  # First image and its keypoints
        images[i + 1], keypoints_list[i + 1],  # Second image and its keypoints
        good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )

    # Display the matches using Matplotlib
    plt.figure(figsize=(12, 6))
    plt.title(f"Feature Matching between Image {i} and Image {i + 1}")
    plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

print("Feature matching visualization complete.")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Fundamental Matrix
import numpy as np
fundamental_matrices = []
for i, match in enumerate(matches):
    pts1 = np.float32([keypoints_list[i][m.queryIdx].pt for m in match])
    pts2 = np.float32([keypoints_list[i + 1][m.trainIdx].pt for m in match])
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
    fundamental_matrices.append(F)

print("Fundamental matrices calculated.")

Fundamental matrices calculated.


In [ ]:
# Camera Pose Recovery
# Replace with the actual intrinsic parameters of your camera or dataset
fx = 704.95  # Example focal length in x
fy = 704.70  # Example focal length in y
cx = 382.04  # Example principal point x
cy = 496.37  # Example principal point y

K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
poses = []

for F in fundamental_matrices:
    E = K.T @ F @ K
    _, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)
    poses.append((R, t))

print("Camera poses recovered.")

Camera poses recovered.


In [ ]:
# Triangulation
points_3d = []

for i, (R, t) in enumerate(poses):
    P1 = K @ np.hstack((np.eye(3), np.zeros((3, 1))))
    P2 = K @ np.hstack((R, t))
    pts4D = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
    points_3d.append(cv2.convertPointsFromHomogeneous(pts4D.T))

print("Triangulation complete.")

Triangulation complete.


In [ ]:
stereo = cv2.StereoSGBM_create(
    minDisparity=0,
    numDisparities=64,
    blockSize=15,
    P1=8 * 3 * 15 ** 2,
    P2=32 * 3 * 15 ** 2,
    disp12MaxDiff=1,
    uniquenessRatio=10,
    speckleWindowSize=100,
    speckleRange=32
)

In [ ]:
# Disparity Maps

# Resize images to the same size
reference_size = images[0].shape[:2]  # (height, width)
images_resized = [cv2.resize(img, (reference_size[1], reference_size[0])) for img in images]

# Create StereoBM matcher

stereo = cv2.StereoBM_create(numDisparities=64, blockSize=15)

# Directory to save disparity maps
output_dir = '/content/drive/MyDrive/DSC 210 Final Project/disparity_maps'
os.makedirs(output_dir, exist_ok=True)

# Generate disparity maps for consecutive pairs
for i in range(len(images_resized) - 1):
    img_left = images_resized[i]
    img_right = images_resized[i + 1]

    # Compute disparity map
    disparity = stereo.compute(img_left, img_right)

    # Normalize disparity for visualization
    disparity_normalized = cv2.normalize(disparity, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    disparity_normalized = cv2.convertScaleAbs(disparity_normalized)

    # Save the disparity map
    output_path = os.path.join(output_dir, f'disparity_map_{i:02d}.png')
    cv2.imwrite(output_path, disparity_normalized)
    print(f"Saved disparity map: {output_path}")

Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_00.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_01.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_02.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_03.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_04.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_05.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_06.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_07.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/disparity_map_08.png
Saved disparity map: /content/drive/MyDrive/DSC 210 Final Project/disparity_maps/d

In [ ]:
# Convert Disparity to Depth Map
focal_length = 704  # Focal length in pixels
baseline = 0.1  # Baseline in meters


disparity_dir = '/content/drive/MyDrive/DSC 210 Final Project/disparity_maps'
depth_output_dir = '/content/drive/MyDrive/DSC 210 Final Project/depth_maps'
os.makedirs(depth_output_dir, exist_ok=True)
# Load disparity images and compute depth maps
disparity_images = sorted([os.path.join(disparity_dir, f) for f in os.listdir(disparity_dir) if f.endswith('.png')])

for i, disparity_path in enumerate(disparity_images):
    disparity = cv2.imread(disparity_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
    # Avoid division by zero
    disparity[disparity == 0] = 1e-6

    # Compute depth
    depth = (focal_length * baseline) / disparity

    # Normalize and save depth map
    depth_normalized = cv2.normalize(depth, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    depth_normalized = np.uint8(depth_normalized)
    depth_path = os.path.join(depth_output_dir, f'depth_map_{i:02d}.png')
    cv2.imwrite(depth_path, depth_normalized)
    print(f"Saved depth map: {depth_path}")


Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_00.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_01.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_02.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_03.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_04.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_05.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_06.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_07.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_08.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_09.png
Saved depth map: /content/drive/MyDrive/DSC 210 Final Project/depth_maps/depth_map_10.png
Saved dept

In [ ]:
# Dense Point Cloud

point_cloud_dir = '/content/drive/MyDrive/DSC 210 Final Project/point_clouds'
os.makedirs(point_cloud_dir, exist_ok=True)

for i, disparity_path in enumerate(disparity_images):
    depth_path = os.path.join(depth_output_dir, f'depth_map_{i:02d}.png')
    depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED).astype(np.float32)

    # Reproject depth map to 3D points
    h, w = depth.shape
    Q = np.float32([[1, 0, 0, -w / 2],
                    [0, -1, 0, h / 2],
                    [0, 0, 0, -focal_length],
                    [0, 0, 1 / baseline, 0]])
    points_3d = cv2.reprojectImageTo3D(depth, Q)

    # Mask invalid points
    mask = depth > 0
    points = points_3d[mask]

    # Save point cloud using Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    point_cloud_path = os.path.join(point_cloud_dir, f'point_cloud_{i:02d}.ply')
    o3d.io.write_point_cloud(point_cloud_path, pcd)
    print(f"Saved point cloud: {point_cloud_path}")


Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_00.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_01.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_02.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_03.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_04.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_05.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_06.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_07.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_08.ply
Saved point cloud: /content/drive/MyDrive/DSC 210 Final Project/point_clouds/point_cloud_09.ply
Saved point cloud: /content/drive/MyDriv